# Исследование надежности заемщиков


In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости вывел медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [22]:
#child1 = data.groupby('children')['debt'].sum() # число должников
#child2 = data.groupby('children')['debt'].count() # число займов
#child_debt_mean = child1 / child2  # поиск конверсии
#print(child1)
#print(child2)
#print(child_debt_mean.apply('{:.2%}'.format)) 

child_debt_mean = data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])
child_debt_mean.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
child_debt_mean['Доля должников'] = child_debt_mean['Доля должников']*100
child_debt_mean

,Всего кредитополучателей,Всего должников,Доля должников
children,,,
0,14091,1063,7.543822
1,4808,444,9.234609
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


Зависимость есть. Чем меньше детей - тем чаще возвращают кредит в срок. 5 детей сложно учитывать, так как данных крайне мало для такого анализа. 4 тоже не много, но вот 1-3 вполне подойдут.

In [26]:
#status1 = data.groupby('family_status')['debt'].sum() # число должников
#status2 = data.groupby('family_status')['debt'].count() # число займов
#family_status_debt_mean = status1 / status2  # поиск конверсии
#print(status1)
#print(status2)
#print(family_status_debt_mean.apply('{:.2%}'.format)) 

family_status_debt_mean  = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count', 'sum', 'mean'])
family_status_debt_mean.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
family_status_debt_mean['Доля должников'] = family_status_debt_mean['Доля должников']*100
family_status_debt_mean.sort_values(by = 'Доля должников', ascending = True)


,Всего кредитополучателей,Всего должников,Доля должников
family_status,,,
вдовец / вдова,951,63,6.624606
в разводе,1189,84,7.064760
женат / замужем,12261,927,7.560558
гражданский брак,4134,385,9.313014
Не женат / не замужем,2796,273,9.763948


Зависимость есть. Как видно из показателей - женатые/замужние чаще выплачивают займы, чем не женатые/не замужние. Так же можно увидеть, что люди в разводе, либо вдовцы чаще всего являются добросовестными заёмщиками. А люди в гражданском браке часто могут оставаться должниками.

In [27]:
#income1 = data.groupby('total_income_category')['debt'].sum() # число должников
#income2 = data.groupby('total_income_category')['debt'].count() # число займов
#income_debt_mean = income1 / income2  # поиск конверсии
#print(income1)
#print(income2)
#print(income_debt_mean.apply('{:.2%}'.format)) 

income_debt_mean = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
income_debt_mean.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
income_debt_mean['Доля должников'] = income_debt_mean['Доля должников']*100
income_debt_mean

,Всего кредитополучателей,Всего должников,Доля должников
total_income_category,,,
A,25,2,8.000000
B,5014,354,7.060231
C,15921,1353,8.498210
D,349,21,6.017192
E,22,2,9.090909


Зависимость есть. Но здесь крайне сложно анализировать с такими маленькими числами. Можно сказать, что чем уровень дохода больше - тем чаще возвращают кредит в срок. Это можно сказать по показателям 'C' и 'B'. Однако показатель 'D' может оспорить данное суждение, его можно принять во внимание, но слишкмо уж заёмщиков с таким уровнем дохода для анализа.

In [28]:
#category1 = data.groupby('purpose_category')['debt'].sum() # число должников
#category2 = data.groupby('purpose_category')['debt'].count() # число займов
#category_debt_mean = category1 / category2  # поиск конверсии
#print(category1)
#print(category2)
#print(category_debt_mean.apply('{:.2%}'.format)) 

category_debt_mean = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
category_debt_mean.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
category_debt_mean['Доля должников'] = category_debt_mean['Доля должников']*100
category_debt_mean.sort_values(by = 'Доля должников', ascending = True)

,Всего кредитополучателей,Всего должников,Доля должников
purpose_category,,,
операции с недвижимостью,10751,780,7.255139
проведение свадьбы,2313,183,7.911803
получение образования,3988,369,9.252758
операции с автомобилем,4279,400,9.347978


Большинство выплаченных займов приходится на недвижимость, причем в категории недвижимости больше всего показателей, что является более точным выводом. Следом идёт категория с займом на свадьбу, данный займ так же чаще выплачивают. Ну а категории с автомобилем и образованием имеют больше всего должников.

#### Возможные причины появления пропусков в исходных данных.

*Ответ:* Причин может быть множество. Например пользователи не внесли данные о себе и тем самым образовались пропуски. Или же какие то данные могли не подтянуться. При сборке данных в таблицу аналитик мог так же совершить ошибку и случайно завести множество пропусков.
Пропуски так же могут быть сделаны умышлено, для приведения пропусков к единому формату.
Так же ответственный за таблицу может сам выбрать вид пропусков(умышленных).